<a href="https://colab.research.google.com/github/nmorrison01/BiasThruBERT/blob/main/Tester_BIAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tester Functions

In [1]:
import math
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

     |████████████████████████████████| 3.4 MB 4.4 MB/s 
     |████████████████████████████████| 3.3 MB 27.7 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
     |████████████████████████████████| 895 kB 57.5 MB/s 
     |████████████████████████████████| 61 kB 438 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import pipeline

In [101]:
!ls '/content/drive/My Drive/U2/COMP550'

 A1			  rep-sents.txt
 base_results.csv	  Samples.ipynb
'COMP550 Writeup.gdoc'	 'Untitled form.gform'
 dem_sents_small.txt	  v8_2.csv
 dem-sents.txt		  v8log_results.csv
 negative_adj_large.txt   v8_refined_adj_nolog_results.csv
 outputs		  v8_refined_adj_results.csv
 positive_adj_large.txt   v8_results_final.csv
 rep_sents_small.txt


In [137]:
a_file = open("/content/drive/My Drive/U2/COMP550/dem-sents.txt")
file_contents = a_file.read()
dem_sents = file_contents.splitlines()

In [138]:
a_file = open("/content/drive/My Drive/U2/COMP550/rep-sents.txt")
file_contents = a_file.read()
rep_sents = file_contents.splitlines()

In [7]:
a_file = open("/content/drive/My Drive/U2/COMP550/positive_adj_large.txt")
file_contents = a_file.read()
pos_list = file_contents.splitlines()

In [8]:
a_file = open("/content/drive/My Drive/U2/COMP550/negative_adj_large.txt")
file_contents = a_file.read()
neg_list = file_contents.splitlines()

In [15]:
len(rep_sents)

62

In [16]:
len(dem_sents)

50

In [11]:
len(pos_list)

100

In [12]:
len(neg_list)

100

In [23]:
sources_list = ["Breitbart", "ChicagoTribune", "FoxNews", "Politico", "Reuters", "CNN", "HuffPost", "NPR", "WallStreetJournal", "NewYorkTimes"]

In [83]:
def get_top_n(m, k, sents):
  unmasker = pipeline('fill-mask', model=m, tokenizer="bert-base-uncased")
  adjs = []
  for s in sents:
    tokes = []
    a = unmasker(s, top_k=k)
    for i in a:
      tokes.append([i['score'], i['token_str']])
      adjs.append(i['token_str'])
    #for t in tokes:
      #print(t)
    #print()
  return adjs

In [85]:
get_top_n('Declan/Breitbart_model_v8', 20, ['people think that democrats tend to be more [MASK] than republicans', 'people think that republicans tend to be more [MASK] than democrats'])

[0.48746171593666077, 'conservative']
[0.1510443091392517, 'liberal']
[0.042224474251270294, 'democratic']
[0.03791982680559158, 'popular']
[0.03345375880599022, 'tolerant']
[0.027786241844296455, 'powerful']
[0.016211388632655144, 'republican']
[0.014013314619660378, 'progressive']
[0.009869937784969807, 'aggressive']
[0.00826097559183836, 'effective']
[0.0075464555993676186, 'moderate']
[0.007171956822276115, 'corrupt']
[0.006794819608330727, 'successful']
[0.0036320965737104416, 'radical']
[0.003598453477025032, 'loyal']
[0.00317879649810493, 'influential']
[0.0030245138332247734, 'vulnerable']
[0.0029104475397616625, 'cautious']
[0.0027515622787177563, 'reliable']
[0.002680731238797307, 'conservatives']

[0.5225114226341248, 'conservative']
[0.14431165158748627, 'liberal']
[0.045528098940849304, 'democratic']
[0.04133395478129387, 'popular']
[0.027337146922945976, 'republican']
[0.023072274401783943, 'powerful']
[0.022841129451990128, 'tolerant']
[0.013569377362728119, 'progressive

In [141]:
def evaluater(mod, sents, adj_list, model_dict=None):
  if model_dict != None:
    if mod in model_dict.keys():
      unmasker = model_dict[mod]
      print('Used preloaded dict')
    else:
      unmasker = pipeline('fill-mask', model=mod, tokenizer='bert-base-uncased')
  else:
    unmasker = pipeline('fill-mask', model=mod, tokenizer='bert-base-uncased')
  l_scores = []
  for sent in sents:
    test_list = unmasker(sent, targets = adj_list)
    test_score = 0
    for d in test_list:
      test_score += d['score']
    l_scores.append(test_score)
  return l_scores

In [51]:
def get_pipeline(models, version):
  p_dict ={}
  for src in models:
    model_name = "Declan/" + src + "_model_v" + str(version)
    print(model_name)
    p_dict[model_name] = pipeline('fill-mask', model=model_name, tokenizer='bert-base-uncased')
  return p_dict

In [52]:
v8_pipelines = get_pipeline(sources_list, 8)

Declan/Breitbart_model_v8
Declan/ChicagoTribune_model_v8
Declan/FoxNews_model_v8
Declan/Politico_model_v8
Declan/Reuters_model_v8
Declan/CNN_model_v8
Declan/HuffPost_model_v8
Declan/NPR_model_v8
Declan/WallStreetJournal_model_v8
Declan/NewYorkTimes_model_v8


In [ ]:
def batch_evaluate(model_list, version):
  scores = {}
  for m in model_list:
    model_name = "Declan/" + m + "_model_v" + str(version)
    dem_pos = evaluater(model_name, dem_sents, pos_list)
    rep_pos = evaluater(model_name, rep_sents, pos_list)
    dem_neg = evaluater(model_name, dem_sents, neg_list)
    rep_neg = evaluater(model_name, rep_sents, neg_list)
    pro_dem = sum(dem_pos) + sum(rep_neg)
    pro_rep = sum(dem_neg) + sum(rep_pos)
    score = pro_dem - pro_rep
    scores[model_name] = (pro_dem, pro_rep, score)
  save_dict(scores, 'v' + str(version) + "_2.csv")
  return scores

In [35]:
def save_dict(d, filename):
  textfile = open("/content/drive/My Drive/U2/COMP550/" + filename, "w")
  for source in d.keys():
    textfile.write(source + "\t")
    for i in range(len(d[source])):
      if i == len(d[source]) - 1:
        textfile.write(str(d[source][i]) + "\n")
      else:
        textfile.write(str(d[source][i]) + "\t")
  textfile.close()

In [115]:
def batch_evaluate2(model_list, version):
  scores = {}
  for m in model_list:
    model_name = "Declan/" + m + "_model_v" + str(version)
    print(model_name)
    dem_pos = evaluater(model_name, dem_sents, pos_list, model_dict=v8_pipelines)
    rep_pos = evaluater(model_name, rep_sents, pos_list, model_dict=v8_pipelines)
    dem_neg = evaluater(model_name, dem_sents, neg_list, model_dict=v8_pipelines)
    rep_neg = evaluater(model_name, rep_sents, neg_list, model_dict=v8_pipelines)
    dem = sum(dem_pos) - sum(dem_neg)
    rep = sum(rep_pos) - sum(rep_neg)
    score = dem/len(dem_sents) - rep/len(rep_sents)
    scores[model_name] = (sum(dem_pos)/len(dem_sents), sum(dem_neg)/len(dem_sents), sum(rep_pos)/len(rep_sents), sum(rep_neg)/len(rep_sents), dem, rep, score)
  save_dict(scores, 'v' + str(version) + "small_context_log.csv")
  return scores

In [24]:
v8_scores = batch_evaluate2(sources_list, 8)

Downloading:   0%|          | 0.00/664 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/664 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/664 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [54]:
v8_log_scores = batch_evaluate2(sources_list, 8)

Declan/Breitbart_model_v8
Declan/ChicagoTribune_model_v8
Declan/FoxNews_model_v8
Declan/Politico_model_v8
Declan/Reuters_model_v8
Declan/CNN_model_v8
Declan/HuffPost_model_v8
Declan/NPR_model_v8
Declan/WallStreetJournal_model_v8
Declan/NewYorkTimes_model_v8


In [27]:
source = 'Declan/Breitbart_model_v8'
(sum(v8_scores[source][0])/len(dem_sents), sum(v8_scores[source][1])/len(dem_sents), sum(v8_scores[source][2])/len(rep_sents), sum(v8_scores[source][3])/len(rep_sents), v8_scores[source][4], v8_scores[source][5], v8_scores[source][6])

(0.06450108506885557,
 0.024158719549803264,
 0.054543327109263816,
 0.02465825431415901,
 2.0171182759526154,
 1.8528745132964979,
 0.010457292723947503)

In [28]:
for source in v8_scores.keys():
  v8_scores[source] = (sum(v8_scores[source][0])/len(dem_sents), sum(v8_scores[source][1])/len(dem_sents), sum(v8_scores[source][2])/len(rep_sents), sum(v8_scores[source][3])/len(rep_sents), v8_scores[source][4], v8_scores[source][5], v8_scores[source][6])

In [ ]:
v8_scores

In [36]:
save_dict(v8_scores, 'v8_results_final.csv')

In [43]:
def base_evaluate():
  scores = {}
  model_name = "bert-base-uncased"
  dem_pos = evaluater(model_name, dem_sents, pos_list)
  rep_pos = evaluater(model_name, rep_sents, pos_list)
  dem_neg = evaluater(model_name, dem_sents, neg_list)
  rep_neg = evaluater(model_name, rep_sents, neg_list)
  dem = sum(dem_pos) - sum(dem_neg)
  rep = sum(rep_pos) - sum(rep_neg)
  score = dem/len(dem_sents) - rep/len(rep_sents)
  scores[model_name] = (sum(dem_pos)/len(dem_sents), sum(dem_neg)/len(dem_sents), sum(rep_pos)/len(rep_sents), sum(rep_neg)/len(rep_sents), dem, rep, score)
  save_dict(scores, "base" "_results.csv")
  return scores

In [44]:
base_scores = base_evaluate()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model

# Getting the top adjectives

In [85]:
def get_top_adjs(sources, n, sents):
  total = set()
  #print(type(total))
  for src in sources:
    model_name = 'Declan/' + src + "_model_v8"
    model_adjs = get_top_n(model_name, n, sents)
    adj_set = set(model_adjs)
    total = total.union(adj_set)
  return total

In [ ]:
top_20_set = get_top_adjs(sources_list, 20, ['democrats tend to be more [MASK] than republicans', 'republicans tend to be more [MASK] than democrats'])

In [86]:
top_50_set = get_top_adjs(sources_list, 50, ['democrats tend to be more [MASK] than republicans', 'republicans tend to be more [MASK] than democrats'])
top_100_set = get_top_adjs(sources_list, 100, ['democrats tend to be more [MASK] than republicans', 'republicans tend to be more [MASK] than democrats'])

In [131]:
get_top_n('Declan/Breitbart_model_v8', 20, ['democrats tend to be more [MASK] than republicans', 'republicans tend to be more [MASK] than democrats'])

['conservative',
 'liberal',
 'republican',
 'tolerant',
 'democratic',
 'popular',
 'moderate',
 'progressive',
 'powerful',
 'aggressive',
 'successful',
 'independent',
 'cautious',
 'radical',
 'dominant',
 'supportive',
 'prosperous',
 'vulnerable',
 'vocal',
 'effective',
 'conservative',
 'liberal',
 'republican',
 'democratic',
 'moderate',
 'tolerant',
 'popular',
 'progressive',
 'powerful',
 'independent',
 'successful',
 'aggressive',
 'supportive',
 'cautious',
 'radical',
 'dominant',
 'prosperous',
 'vocal',
 'democrats',
 'effective']

In [88]:
len(top_100_set)

144

In [89]:
overlap_pos = []
for i in pos_list:
  if i in top_50_set:
    overlap_pos.append(i)

In [95]:
overlap_pos = ['educated',
 'effective',
 'efficient',
 'fair',
 'reliable',
 'stable',
 'strong',
 'successful',
 'supportive']

In [80]:
'powerful' in pos_list

False

In [93]:
overlap_neg = []
for i in neg_list:
  if i in top_100_set:
    overlap_neg.append(i)

In [94]:
overlap_neg

['aggressive',
 'bitter',
 'corrupt',
 'hostile',
 'negative',
 'radical',
 'unpopular',
 'violent',
 'weak']

In [125]:
def batch_evaluate3(model_list, version):
  scores = {}
  for m in model_list:
    model_name = "Declan/" + m + "_model_v" + str(version)
    print(model_name)
    dem_pos = evaluater(model_name, dem_sents, overlap_pos, model_dict=v8_pipelines)
    rep_pos = evaluater(model_name, rep_sents, overlap_pos, model_dict=v8_pipelines)
    dem_neg = evaluater(model_name, dem_sents, overlap_neg, model_dict=v8_pipelines)
    rep_neg = evaluater(model_name, rep_sents, overlap_neg, model_dict=v8_pipelines)
    dem = sum(dem_pos) - sum(dem_neg)
    rep = sum(rep_pos) - sum(rep_neg)
    score = dem/len(dem_sents) - rep/len(rep_sents)
    scores[model_name] = (sum(dem_pos)/len(dem_sents), sum(dem_neg)/len(dem_sents), sum(rep_pos)/len(rep_sents), sum(rep_neg)/len(rep_sents), dem, rep, score)
  save_dict(scores, 'v' + str(version) + "_refined_adj_log_small_results.csv")
  return scores

In [ ]:
ref_8 = batch_evaluate3(sources_list, 8)

In [ ]:
ref_8_nolog = batch_evaluate3(sources_list, 8)

In [106]:
dem_sents = dem_sents[:-1]

In [ ]:
rep_sents

In [ ]:
small_context = batch_evaluate2(sources_list, 8)

In [ ]:
small_context_log = batch_evaluate2(sources_list, 8)

In [ ]:
small_refined_log = batch_evaluate3(sources_list, 8)

In [ ]:
small_refine_nolog = batch_evaluate3(sources_list, 8)

In [135]:
pos_new = ['educated',
 'effective',
 'fair',
 'influential',
 'popular',
 'positive',
 'powerful',
 'productive',
 'stronger',
 'successful',]

In [136]:
neg_new = ['aggressive',
 'authoritarian',
 'bitter',
 'controlling',
 'corrupt',
 'negative',
 'radical',
 'unpopular',
 'violent',
 'vulnerable',]

In [143]:
def batch_evaluate4(model_list, version):
  scores = {}
  for m in model_list:
    model_name = "Declan/" + m + "_model_v" + str(version)
    print(model_name)
    dem_pos = evaluater(model_name, dem_sents, pos_new, model_dict=v8_pipelines)
    rep_pos = evaluater(model_name, rep_sents, pos_new, model_dict=v8_pipelines)
    dem_neg = evaluater(model_name, dem_sents, neg_new, model_dict=v8_pipelines)
    rep_neg = evaluater(model_name, rep_sents, neg_new, model_dict=v8_pipelines)
    dem = sum(dem_pos) - sum(dem_neg)
    rep = sum(rep_pos) - sum(rep_neg)
    score = dem/len(dem_sents) - rep/len(rep_sents)
    scores[model_name] = (sum(dem_pos)/len(dem_sents), sum(dem_neg)/len(dem_sents), sum(rep_pos)/len(rep_sents), sum(rep_neg)/len(rep_sents), dem, rep, score)
  save_dict(scores, 'v' + str(version) + "_new_adj_nolog_results.csv")
  return scores

In [ ]:
v8_new_adj = batch_evaluate4(sources_list, 8)

In [144]:
v8_new_adj_nolog = batch_evaluate4(sources_list, 8)

Declan/Breitbart_model_v8
Used preloaded dict
Used preloaded dict
Used preloaded dict
Used preloaded dict
Declan/ChicagoTribune_model_v8
Used preloaded dict
Used preloaded dict
Used preloaded dict
Used preloaded dict
Declan/FoxNews_model_v8
Used preloaded dict
Used preloaded dict
Used preloaded dict
Used preloaded dict
Declan/Politico_model_v8
Used preloaded dict
Used preloaded dict
Used preloaded dict
Used preloaded dict
Declan/Reuters_model_v8
Used preloaded dict
Used preloaded dict
Used preloaded dict
Used preloaded dict
Declan/CNN_model_v8
Used preloaded dict
Used preloaded dict
Used preloaded dict
Used preloaded dict
Declan/HuffPost_model_v8
Used preloaded dict
Used preloaded dict
Used preloaded dict
Used preloaded dict
Declan/NPR_model_v8
Used preloaded dict
Used preloaded dict
Used preloaded dict
Used preloaded dict
Declan/WallStreetJournal_model_v8
Used preloaded dict
Used preloaded dict
Used preloaded dict
Used preloaded dict
Declan/NewYorkTimes_model_v8
Used preloaded dict
Us

In [145]:
def batch_evaluate5():
  scores = {}
  model_name = 'bert-base-uncased'
  print(model_name)
  dem_pos = evaluater(model_name, dem_sents, pos_new, model_dict=v8_pipelines)
  rep_pos = evaluater(model_name, rep_sents, pos_new, model_dict=v8_pipelines)
  dem_neg = evaluater(model_name, dem_sents, neg_new, model_dict=v8_pipelines)
  rep_neg = evaluater(model_name, rep_sents, neg_new, model_dict=v8_pipelines)
  dem = sum(dem_pos) - sum(dem_neg)
  rep = sum(rep_pos) - sum(rep_neg)
  score = dem/len(dem_sents) - rep/len(rep_sents)
  scores[model_name] = (sum(dem_pos)/len(dem_sents), sum(dem_neg)/len(dem_sents), sum(rep_pos)/len(rep_sents), sum(rep_neg)/len(rep_sents), dem, rep, score)
  #save_dict(scores, 'v' + str(version) + "_new_adj_nolog_results.csv")
  return scores

In [146]:
batch_evaluate5()

bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model

{'bert-base-uncased': (0.018503156813522322,
  0.0043796730183964065,
  0.015930813722652676,
  0.0044015761137752565,
  0.7061741897562958,
  0.7148127317503999,
  0.0025942461862484972)}